# Feature Transformation with Amazon a SageMaker Processing Job and Scikit-Learn

In this notebook, we convert raw text into BERT embeddings.  This will allow us to perform natural language processing tasks such as text classification.

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Scikit-Learn are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Scikit-Learn in a managed SageMaker environment to run our processing workload.

# NOTE:  THIS NOTEBOOK WILL TAKE A 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


## Contents

1. Setup Environment
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Data

# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Setup Input Data

In [ ]:
%store -r s3_public_path_tsv

In [ ]:
try:
    s3_public_path_tsv
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the INGEST section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
print(s3_public_path_tsv)

In [ ]:
%store -r s3_private_path_tsv

In [ ]:
try:
    s3_private_path_tsv
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the INGEST section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
print(s3_private_path_tsv)

# Let's Copy 1 More Large Data File to Use For Training

In [ ]:
!aws s3 cp --recursive $s3_public_path_tsv/ $s3_private_path_tsv/ --exclude "*" --include "amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"

In [ ]:
raw_input_data_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(raw_input_data_s3_uri)

In [ ]:
!aws s3 ls $raw_input_data_s3_uri

# Run the Processing Job using Amazon SageMaker

Next, use the Amazon SageMaker Python SDK to submit a processing job using our custom python script.

# Review the Processing Script

In [ ]:
!pygmentize preprocess-scikit-text-to-bert.py

Run this script as a processing job.  You also need to specify one `ProcessingInput` with the `source` argument of the Amazon S3 bucket and `destination` is where the script reads this data from `/opt/ml/processing/input` (inside the Docker container.)  All local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to.  For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`.  You also give the `ProcessingOutput` value for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The arguments parameter in the `run()` method are command-line arguments in our `preprocess-scikit-text-to-bert.py` script.

Note that we sharding the data using `ShardedByS3Key` to spread the transformations across all worker nodes in the cluster.

# Track the `Experiment`
We will track every step of this experiment throughout the `prepare`, `train`, `optimize`, and `deploy`.

# Concepts

**Experiment**: A collection of related Trials.  Add Trials to an Experiment that you wish to compare together.

**Trial**: A description of a multi-step machine learning workflow. Each step in the workflow is described by a Trial Component. There is no relationship between Trial Components such as ordering.

**Trial Component**: A description of a single step in a machine learning workflow. For example data cleaning, feature extraction, model training, model evaluation, etc.

**Tracker**: A logger of information about a single TrialComponent.

<img src="img/sagemaker-experiments.png" width="90%" align="left">


# Create the `Experiment`

In [ ]:
import time
from smexperiments.experiment import Experiment

timestamp = int(time.time())

experiment = Experiment.create(
                experiment_name='Amazon-Customer-Reviews-BERT-Experiment-{}'.format(timestamp),
                description='Amazon Customer Reviews BERT Experiment', 
                sagemaker_boto_client=sm)

experiment_name = experiment.experiment_name
print('Experiment name: {}'.format(experiment_name))

# Create the `Trial`

In [ ]:
import time
from smexperiments.trial import Trial

timestamp = int(time.time())

trial = Trial.create(trial_name='trial-{}'.format(timestamp),
                     experiment_name=experiment_name,
                     sagemaker_boto_client=sm)

trial_name = trial.trial_name
print('Trial name: {}'.format(trial_name))

# Create the `Experiment Config`

In [ ]:
experiment_config = {
    'ExperimentName': experiment_name,
    'TrialName': trial.trial_name,
    'TrialComponentDisplayName': 'prepare'
}

In [ ]:
print(experiment_name)

In [ ]:
%store experiment_name

In [ ]:
print(trial_name)

In [ ]:
%store trial_name

# Set the Processing Job Hyper-Parameters 

In [ ]:
processing_instance_type='ml.c5.2xlarge'
processing_instance_count=1
train_split_percentage=0.90
validation_split_percentage=0.05
test_split_percentage=0.05
balance_dataset=True
max_seq_length=64

# Choosing a `max_seq_length` for BERT
Since a smaller `max_seq_length` leads to faster training and lower resource utilization, we want to find the smallest review length that captures `70%` of our reviews.

Remember our distribution of review lengths from a previous section?

```
mean         67.930174
std         130.954079
min           1.000000
10%           4.000000
20%          14.000000
30%          21.000000
40%          25.000000
50%          31.000000
60%          42.000000
70%          59.000000
80%          87.000000
90%         149.000000
100%       5347.000000
max        5347.000000
```

![](img/review_word_count_distribution.png)

Review length `59` represents the `70th` percentile for this dataset.  However, it's best to stick with powers-of-2 when using BERT.  So let's choose `64` as this is the smallest power-of-2 greater than `59`.  Reviews with length > `64` will be truncated to `64`.

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(framework_version='0.23-1',
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=processing_instance_count,
                             max_runtime_in_seconds=7200)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor.run(code='preprocess-scikit-text-to-bert.py',
              inputs=[
                    ProcessingInput(source=raw_input_data_s3_uri,
                                    destination='/opt/ml/processing/input/data/',
                                    s3_data_distribution_type='ShardedByS3Key')
              ],
              outputs=[
                    ProcessingOutput(s3_upload_mode='EndOfJob',
                                     output_name='bert-train',
                                     source='/opt/ml/processing/output/bert/train'),
                    ProcessingOutput(s3_upload_mode='EndOfJob',
                                     output_name='bert-validation',
                                     source='/opt/ml/processing/output/bert/validation'),
                    ProcessingOutput(s3_upload_mode='EndOfJob',
                                     output_name='bert-test',
                                     source='/opt/ml/processing/output/bert/test'),
              ],
              arguments=['--train-split-percentage', str(train_split_percentage),
                         '--validation-split-percentage', str(validation_split_percentage),
                         '--test-split-percentage', str(test_split_percentage),
                         '--max-seq-length', str(max_seq_length),
                         '--balance-dataset', str(balance_dataset)
              ],
              experiment_config=experiment_config,
              logs=True,
              wait=False)

In [ ]:
scikit_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']
print(scikit_processing_job_name)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, scikit_processing_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, scikit_processing_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, scikit_processing_job_name, region)))


# Monitor the Processing Job

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=scikit_processing_job_name,
                                                                            sagemaker_session=sess)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
processing_job_name = processing_job_description['ProcessingJobName']
print(processing_job_name)

In [ ]:
running_processor.wait(logs=False)

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output Data

Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [ ]:
processing_job_description = running_processor.describe()

output_config = processing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'bert-train':
        processed_train_data_s3_uri = output['S3Output']['S3Uri']
    if output['OutputName'] == 'bert-validation':
        processed_validation_data_s3_uri = output['S3Output']['S3Uri']
    if output['OutputName'] == 'bert-test':
        processed_test_data_s3_uri = output['S3Output']['S3Uri']
        
print(processed_train_data_s3_uri)
print(processed_validation_data_s3_uri)
print(processed_test_data_s3_uri)

In [ ]:
!aws s3 ls $processed_train_data_s3_uri/

In [ ]:
!aws s3 ls $processed_validation_data_s3_uri/

In [ ]:
!aws s3 ls $processed_test_data_s3_uri/

# Pass Variables to the Next Notebook(s)

In [ ]:
%store processing_job_name

In [ ]:
%store processed_train_data_s3_uri

In [ ]:
%store processed_validation_data_s3_uri

In [ ]:
%store processed_test_data_s3_uri

In [ ]:
%store max_seq_length

In [ ]:
%store experiment_name

In [ ]:
%store trial_name

In [ ]:
%store

# Show the Experiment Tracking Lineage

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

import pandas as pd
pd.set_option("max_colwidth", 500)
#pd.set_option("max_rows", 100)

experiment_analytics = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    sort_by="CreationTime",
    sort_order="Descending"
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

In [ ]:
trial_component_name=experiment_analytics_df.TrialComponentName[0]
print(trial_component_name)

In [ ]:
trial_component_description=sm.describe_trial_component(TrialComponentName=trial_component_name)
trial_component_description

In [ ]:
from sagemaker.lineage.visualizer import LineageTableVisualizer

lineage_table_viz = LineageTableVisualizer(sess)
lineage_table_viz_df = lineage_table_viz.show(processing_job_name=processing_job_name)
lineage_table_viz_df

In [ ]:
# from sagemaker.lineage.visualizer import LineageTableVisualizer

# lineage_table_viz = LineageTableVisualizer(sess)
# lineage_table_viz_df = lineage_table_viz.show(trial_component_name=trial_component_name)
# lineage_table_viz_df

In [ ]:
from sagemaker.analytics import ArtifactAnalytics

artifact_analytics = ArtifactAnalytics()

artifacts_df = artifact_analytics.dataframe()
artifacts_df

In [ ]:
# from sagemaker.lineage.association import Association
# from sagemaker.lineage.artifact import Artifact

# incoming_associations = Association.list(destination_arn='arn:aws:sagemaker:us-east-1:835319576252:artifact/ebfc31f9f6007feb1a70daaf24339c08',
#                                 sort_by='CreationTime', 
#                                 sort_order='Descending')

# for association in enumerate(incoming_associations):
#     print(association)

In [ ]:
# from sagemaker.lineage.association import Association
# from sagemaker.lineage.artifact import Artifact

# incoming_associations = Association.list(source_arn='arn:aws:sagemaker:us-east-1:835319576252:artifact/ebfc31f9f6007feb1a70daaf24339c08',
#                                 sort_by='CreationTime', 
#                                 sort_order='Descending')

# for association in enumerate(incoming_associations):
#     print(association)

In [ ]:
# # list all the contexts
# from sagemaker.lineage.context import Context

# contexts = Context.list(sort_by='CreationTime', 
#                         sort_order='Descending')

# for ctx in contexts:
#     print(ctx.source.source_uri)

In [ ]:
# from sagemaker.lineage.action import Action

# actions = Action.list(sort_by='CreationTime', sort_order='Descending')

# for action in actions:
#     print(action)

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();